In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rd

cols = ['Underlying']
strikeRange = [455,456,457,458,459,460,461]

for i in range(len(strikeRange)):
    cols += [f'Strike: ${strikeRange[i]}', f'call_openint {strikeRange[i]}', f'call_volume {strikeRange[i]}', f'call_bid {strikeRange[i]}',f'call_ask {strikeRange[i]}',f'put_bid {strikeRange[i]}',f'put_ask {strikeRange[i]}',f'put_volume {strikeRange[i]}',f'put_openint {strikeRange[i]}']

df = pd.DataFrame(columns=cols)


num_trades = 1500
for t in range(num_trades):
    if t > 0:
        df.loc[t,'Underlying'] = df.loc[t-1,'Underlying'] + np.random.normal(0,5)
    else:
        df.loc[t,'Underlying'] = 458
        for strike in range(len(strikeRange)):
            df.loc[t,f'call_openint {strikeRange[strike]}'] = round(np.random.normal(100,30))
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(400 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            
        
    for strike in range(len(strikeRange)):
        df.loc[t,f'Strike: ${strikeRange[strike]}'] = f'Strike: ${strikeRange[strike]}'

        #set volume and open interest
        if t>0:
            df.loc[t,f'call_openint {strikeRange[strike]}'] = max(df.loc[t-1,f'call_openint {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            # if OI jumps and vol doesn't, market responds heavily
            # if OI jumps and vol jumps, market responds medium
            # if OI doesn't jump and vol does, market responds medium
            # if neither jump, status quo
            old_cOI = df.loc[t-1,f'call_openint {strikeRange[strike]}']
            old_pOI = df.loc[t-1,f'put_openint {strikeRange[strike]}']
            old_cVol = df.loc[t-1,f'call_volume {strikeRange[strike]}']
            old_pVol = df.loc[t-1,f'put_volume {strikeRange[strike]}']

            cOI = df.loc[t,f'call_openint {strikeRange[strike]}']
            pOI = df.loc[t,f'put_openint {strikeRange[strike]}']
            cVol = df.loc[t,f'call_volume {strikeRange[strike]}']
            pVol = df.loc[t,f'put_volume {strikeRange[strike]}']

            # if cOI > old_cOI + 5 and cVol < old_cVol + 5:
            #     # call jumps, put drops
            # elif pOI > old_pOI + 5 and pVol < old_pVol + 5:
            #     # call drops, put jumps
            # elif c
            #get call data
            call_bid = np.random.normal(2,1)
            call_ask = call_bid-np.random.normal(0.2,0.1)
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            #calculate put call parity to back out put bid/ask
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

            #get put data
            put_bid = mean_put+np.random.normal(0.2,0.1)
            put_ask = mean_put-np.random.normal(0.2,0.1)
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask
        
        else: #initial conditions
            #get call data
            call_bid = np.random.normal(2,1)
            call_ask = call_bid-np.random.normal(0.2,0.1)
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            #calculate put call parity to back out put bid/ask
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

            #get put data
            put_bid = mean_put+np.random.normal(0.2,0.1)
            put_ask = mean_put-np.random.normal(0.2,0.1)
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask

        

display(df)
df.to_csv('OptionsData1.csv')

,Underlying,Strike: $455,call_openint 455,call_volume 455,call_bid 455,call_ask 455,put_bid 455,put_ask 455,put_volume 455,put_openint 455,...,put_openint 460,Strike: $461,call_openint 461,call_volume 461,call_bid 461,call_ask 461,put_bid 461,put_ask 461,put_volume 461,put_openint 461
0,458,Strike: $455,21,179,0.223185,-0.013855,-2.611554,-3.120778,0,379,...,262,Strike: $461,103,97,1.640133,1.537351,4.812437,4.346933,0,297
1,459.515961,Strike: $455,18,132,-1.310931,-1.553713,-5.664504,-6.337943,0,182,...,73,Strike: $461,107,43,2.149963,1.836926,3.5526,3.340981,57,93
2,459.302734,Strike: $455,10,140,0.707126,0.578394,-3.279817,-3.859498,0,190,...,81,Strike: $461,106,44,0.654001,0.529669,2.26256,2.05542,56,94
3,455.7713,Strike: $455,0,150,2.41537,2.209676,1.685314,1.210985,0,200,...,82,Strike: $461,94,56,3.532966,3.383574,8.884906,8.531876,44,106
4,463.410331,Strike: $455,10,140,3.08951,2.991063,-5.114881,-5.384076,0,190,...,83,Strike: $461,97,53,1.843715,1.734238,-0.528492,-0.64496,47,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,511.922353,Strike: $455,891,0,2.116537,1.963322,-54.731672,-55.108964,150,0,...,0,Strike: $461,5,145,1.245991,1.024994,-49.497732,-49.946457,0,195
1496,514.795865,Strike: $455,879,0,0.418646,0.278113,-59.084093,-59.542806,150,0,...,0,Strike: $461,0,150,3.108383,2.894908,-50.619367,-51.043495,0,200
1497,510.954277,Strike: $455,881,0,0.978547,0.844363,-54.751056,-55.317069,150,0,...,0,Strike: $461,0,150,3.726458,3.521042,-46.118602,-46.504036,0,200
1498,509.860259,Strike: $455,892,0,1.55471,1.386149,-53.149207,-53.836381,150,0,...,0,Strike: $461,0,150,2.017034,1.932777,-46.609753,-46.915055,0,200
